## Loading dependencies

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
import gc

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
# This package is used for finding longest common subsequence between two strings
# you can write your own dp code for this
# import distance
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
# from fuzzywuzzy import fuzz
from sklearn.manifold import TSNE
# Import the Required lib packages for WORD-Cloud generation
# https://stackoverflow.com/questions/45625434/how-to-install-wordcloud-in-python3-6
from wordcloud import WordCloud, STOPWORDS
from os import path
from PIL import Image
os.chdir('/Users/subham/Desktop')

### Problem Statement
- We have got a huge pandas data frame, and we want to apply a complex function to it which takes a lot of time.

## Utility function for preprocessing data

In [7]:
def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    
    porter = PorterStemmer()
    pattern = re.compile('\W')
    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
    
    
    if type(x) == type(''):
        x = porter.stem(x)
        example1 = BeautifulSoup(x)
        x = example1.get_text()
               
    
    return x

## Pandas apply() method

In [2]:
#https://www.kaggle.com/c/quora-question-pairs/data?select=train.csv.zip
df  = pd.read_csv("train.csv")

In [3]:
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [4]:
df.shape

(404290, 6)

In [6]:
import re
from tqdm import tqdm
tqdm.pandas()
def extract_features(df):
    # preprocessing each question
    df["question1"] = df["question1"].fillna("").progress_apply(preprocess)
    return df

In [8]:
import datetime

start = datetime.datetime.now()
df = extract_features(df)
end = datetime.datetime.now()

print("-"*44)
print("Execution Time: {} ".format(end-start))
print("-"*44)

100%|██████████| 404290/404290 [02:34<00:00, 2619.26it/s]

--------------------------------------------
Execution Time: 0:02:34.442627 
--------------------------------------------


### Why apply method is slow ?
- when used with a large dataset running with a for loop under the hood and overhead of IO operations makes it slow

## SWIFTER library
#### As per their documentation it defines Swifter as:

- A package which efficiently applies any function to a pandas dataframe or series in the fastest available manner

In [20]:
import swifter
df  = pd.read_csv("train.csv")


In [11]:
import re
from tqdm import tqdm
tqdm.pandas()
def extract_features(df):
    # preprocessing each question
    df["question1"] = df["question1"].fillna("").swifter.apply(preprocess)
    return df

In [12]:
import datetime

start = datetime.datetime.now()
df = extract_features(df)
end = datetime.datetime.now()

print("-"*44)
print("Execution Time: {} ".format(end-start))
print("-"*44)


--------------------------------------------
Execution Time: 0:02:32.912511 
--------------------------------------------


__In this particular case, Swifter is using Dask to parallelize our apply functions with the default value of npartitions = cpu_count()*2.__

#### npartitions(12)

In [21]:
import swifter
df  = pd.read_csv("train.csv")


In [22]:
import re
from tqdm import tqdm
tqdm.pandas()
def extract_features(df):
    # preprocessing each question
    df["question1"] = df["question1"].fillna("").swifter.set_npartitions(12).apply(preprocess)
    return df

In [23]:
import datetime

start = datetime.datetime.now()
df = extract_features(df)
end = datetime.datetime.now()

print("-"*44)
print("Execution Time: {} ".format(end-start))
print("-"*44)


--------------------------------------------
Execution Time: 0:02:28.555410 
--------------------------------------------


#### npartitions(24)

In [27]:
import swifter
df  = pd.read_csv("train.csv")



In [25]:
import re
from tqdm import tqdm
tqdm.pandas()
def extract_features(df):
    # preprocessing each question
    df["question1"] = df["question1"].fillna("").swifter.set_npartitions(24).apply(preprocess)
    return df

In [26]:
import datetime

start = datetime.datetime.now()
df = extract_features(df)
end = datetime.datetime.now()

print("-"*44)
print("Execution Time: {} ".format(end-start))
print("-"*44)


--------------------------------------------
Execution Time: 0:02:26.554151 
--------------------------------------------


### Parallel computing 
__is a task where a large chunk of data is divided into smaller parts and 
processed simultaneously using the modern hardware capability of multiple CPU’s and cores of the machine.__

### Dask is a library for parallel computing in Python and it is basically used for the following two tasks:

1. Task Scheduler: It is used for optimizing the task scheduling jobs .
2. Store the data in Parallel Arrays, Dataframe and it runs on top of task scheduler

In [12]:
import dask.dataframe as dd
df  = pd.read_csv("train.csv")

ddf = dd.from_pandas(df, npartitions=-1)

In [13]:
ddf = ddf.map_partitions(extract_features).compute()

### mapply library

In [8]:
from multiprocessing import  Pool
import numpy as np
import os
import json
import pandas as pd
import tqdm
import datetime
import time
import mapply

import time


df  = pd.read_csv("train.csv")
mapply.init(n_workers=-1)

In [9]:
import re
from tqdm import tqdm
tqdm.pandas()
def extract_features(df):
    # preprocessing each question
    df["question1"] = df["question1"].fillna("").mapply(preprocess)
    return df

In [10]:
import datetime

start = datetime.datetime.now()
df = extract_features(df)
end = datetime.datetime.now()

print("-"*44)
print("Execution Time: {} ".format(end-start))
print("-"*44)


--------------------------------------------
Execution Time: 0:01:12.366082 
--------------------------------------------


In [15]:
!system_profiler SPHardwareDataType; 

Hardware:

    Hardware Overview:

      Model Name: MacBook Air
      Model Identifier: MacBookAir7,2
      Processor Name: Dual-Core Intel Core i5
      Processor Speed: 1.8 GHz
      Number of Processors: 1
      Total Number of Cores: 2
      L2 Cache (per Core): 256 KB
      L3 Cache: 3 MB
      Hyper-Threading Technology: Enabled
      Memory: 8 GB
      Boot ROM Version: 195.0.0.0.0
      SMC Version (system): 2.27f2
      Serial Number (system): FVFZ6GAAJ1WK
      Hardware UUID: FF1D5F2B-E382-5003-BFDE-8A342AD6B4E2



### Reference
- https://towardsdatascience.com/make-your-own-super-pandas-using-multiproc-1c04f41944a1
- https://stackoverflow.com/questions/45545110/make-pandas-dataframe-apply-use-all-cores